# Data Normalization Mini Class

In [1]:
import os

import pandas as pd
import sqlite3

from data_norm_class.constants import (
    FILE_DATA_RAW,
    FOLDER_DATA_NORMALIZED
)
from data_norm_class.io import (
    read_raw_aarc_data
)

In [7]:
FOLDER_DATA_NORMALIZED = os.path.join("..", FOLDER_DATA_NORMALIZED)
FILE_DATA_RAW = os.path.join("..", FILE_DATA_RAW)